In [1]:
import numpy as np
import os
import cv2
import os, glob
import os.path
image_row = 224
image_col = 224
frame = 32

# input for 3D CNN
def video3d(filename,image_row,image_col,frame):
    cap = cv2.VideoCapture(filename)
    nframe = cap.get(cv2.CAP_PROP_FRAME_COUNT)

    frames = [x for x in range(frame)]
    framearray = []

    for i in range(frame):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frames[i])
        ret, frame = cap.read()
        frame = cv2.resize(frame, (image_row, image_col))
        framearray.append(frame)
    cap.release()
    return np.array(framearray)

In [2]:
emotion_class = ['Exciting','Fear','Neutral','Relaxation','Sad','Tension']
X = []
y = []
i = 0
# video X input과 y input 생성
for emotion in emotion_class:
    targerdir = r'Music_Video_Emotion_Dataset-master/' + emotion
    files = os.listdir(targerdir)
    for video in files:
        file_name = targerdir+'/'+video
        arr=video3d(file_name,image_row,image_col,frame)
        X.append(arr)
        y.append(i)
    i+=1

In [3]:
X=np.array(X)

In [4]:
X.shape

(1979, 32, 112, 112, 3)

In [5]:
from keras.utils import np_utils
Y = np_utils.to_categorical(y)

In [6]:
np.savez('video_dataset.npz',X=X,Y=Y)

In [7]:
saved_npz  = np.load('video_dataset.npz')

In [8]:
X=saved_npz['X']

In [9]:
y = saved_npz['Y']

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, shuffle=True, random_state=1004)

In [12]:
np.savez('c3d_video_dataset_train.npz',X=X_train,Y=y_train)
np.savez('c3d_video_dataset_test.npz',X=X_test,Y=y_test)